In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import re

In [2]:
xls = pd.ExcelFile("Product Matching Dataset.xlsx")

In [3]:
master_df = pd.read_excel(xls, "Master File")
dataset_df = pd.read_excel(xls, "Dataset")

In [4]:
dataset_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [5]:
master_df.head()

,sku,product_name,product_name_ar,price
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0


In [6]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83562 entries, 0 to 83561
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sku                          83562 non-null  int64  
 1   marketplace_product_name_ar  83562 non-null  object 
 2   seller_item_name             83562 non-null  object 
 3   price                        83562 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.6+ MB


## 1. Data Preprocessing

### a. Cleaning

In [7]:
# remove punctuation and convert to lowercase
dataset_df['seller_item_name'] = dataset_df['seller_item_name'].map(lambda s: re.sub(r'[^\w\s]','',s.lower()))

#### Generating Dictionary from master file

In [14]:
master_df.duplicated().sum()

0

In [65]:
master_df.isna().sum()

sku                      0
product_name             0
product_name_ar          0
price                    0
product_name_clean       0
product_name_ar_clean    0
dtype: int64

In [76]:
def normalize_english(text):
    text = str(text).lower()
    # remove numbers (for master file only)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) # remove non-alphanumeric characters
    # text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = text.strip()
    return text


def normalize_arabic(text):
    # remove numbers (for master file only)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r"[\u064B-\u065F]", "", text)  # Remove Arabic diacritics (tashkeel)
    # text = re.sub(r'[^\u0621-\u064A\s]', '', text) # remove non-arabic characters
    text = re.sub(r'[^\u0621-\u064A0-9\s]', ' ', text) # remove non-arabic characters except for numbers
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = str(text).strip()
    return text

In [62]:
normalize_english(" we.ork//  / Hello,World!  ")

'we ork hello world'

In [71]:
normalize_arabic("helloo فيدروب 2800 وحدة دولية//مل نقط بالفم helloo 15 مل")

'فيدروب وحدة دولية مل نقط بالفم مل'

In [ ]:
master_df['product_name_clean'] = master_df['product_name'].map(normalize_english)
master_df['product_name_ar_clean'] = master_df['product_name_ar'].map(normalize_arabic)

In [73]:
master_df.head()

,sku,product_name,product_name_ar,price,product_name_clean,product_name_ar_clean
0,279,ANAFRONIL 75 MG 20 TAB,انافرونيل 75 مجم اس ار 20 قرص,75.0,anafronil mg tab,انافرونيل مجم اس ار قرص
1,2282,LOPRECOUGH SYRUP 100 ML,لوبريكاف شراب 100 مل,28.5,loprecough syrup ml,لوبريكاف شراب مل
2,4331,TOMEX PLUS 50 TAB,تومكس بلس 50 قرص,60.0,tomex plus tab,تومكس بلس قرص
3,1022,TAROLIMUS 0.03% OINT. 15 GM,تاروليمس 0.03 % مرهم 15 جم,129.0,tarolimus oint gm,تاروليمس مرهم جم
4,116,GLIPTUS PLUS 50/1000 MG 30 TAB,جليبتس بلس 50/1000 مجم 30 قرص,192.0,gliptus plus mg tab,جليبتس بلس مجم قرص


In [83]:
from collections import Counter

english_words = Counter(" ".join(master_df['product_name_clean']).split())
arabic_words = Counter(" ".join(master_df['product_name_ar_clean']).split())

# filter out one-letter words
english_words = {k:v for k,v in english_words.items() if len(k) > 1}
arabic_words = {k:v for k,v in arabic_words.items() if len(k) > 1}

english_words = Counter(english_words)
arabic_words = Counter(arabic_words)

In [85]:
english_words

Counter({'mg': 614,
         'tab': 509,
         'ml': 248,
         'cap': 129,
         'gm': 86,
         'syrup': 44,
         'plus': 38,
         'amp': 37,
         'eye': 37,
         'cream': 36,
         'drops': 35,
         'susp': 27,
         'sachets': 18,
         'mcg': 16,
         'milk': 16,
         'spray': 15,
         'nasal': 14,
         'drop': 13,
         'vial': 12,
         'oint': 10,
         'oral': 10,
         'dose': 10,
         'erastapex': 9,
         'co': 9,
         'baby': 9,
         'solution': 8,
         'gel': 8,
         'adult': 8,
         'xr': 8,
         'with': 8,
         'advance': 8,
         'limitless': 8,
         'mr': 7,
         'panadol': 6,
         'vaginal': 6,
         'empacoza': 6,
         'lozenges': 6,
         'syringe': 6,
         'hero': 6,
         'tareg': 6,
         'pre': 6,
         'filled': 6,
         'voltaren': 6,
         'supp': 5,
         'extra': 5,
         'conventin': 5,
         'sachet'

In [99]:
def save_symspell_dict(words_dict, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for word, freq in words_dict.items():
            f.write(f"{word}\t{freq}\n")

In [100]:
save_symspell_dict(english_words, "symspell_english.txt")
save_symspell_dict(arabic_words, "symspell_arabic.txt")

In [105]:
from symspellpy import SymSpell, Verbosity

sym_spell_en = SymSpell()
sym_spell_ar = SymSpell()
sym_spell_en.load_dictionary("symspell_english.txt", term_index=0, count_index=1)
sym_spell_ar.load_dictionary("symspell_arabic.txt", term_index=0, count_index=1, encoding="utf-8")

True

In [113]:
def correct_spelling(text):
    suggestions = sym_spell_ar.lookup(text, Verbosity.CLOSEST, max_edit_distance=2)
    if suggestions:
        return suggestions[0].term
    return text

In [114]:
dataset_df['seller_item_name_clean'] = dataset_df['seller_item_name'].map(correct_spelling)

In [115]:
correct_spelling("ثروومباااكس")

'ثروومباااكس'

In [96]:
dataset_df.sample(10)

,sku,marketplace_product_name_ar,seller_item_name,price,seller_item_name_clean
77701,1844,سويتال 1 جم 75 كيس,سويتال75 كيس,85.0,سويتال75 كيس
73761,3770,اكتوفنت سي آر 4 مجم 20 قرص,اكتوفنت سى ار 4مجم 20قرص,12.0,اكتوفنت سى ار 4مجم 20قرص
6735,399,اتور 40 مجم 10 قرص,اتور40م ق,80.0,اتور40م ق
70737,1022,تاروليمس 0.03 % مرهم 15 جم,تاروليمس مرهم 15 جم سعر جديد,129.0,تاروليمس مرهم 15 جم سعر جديد
78708,2285,جراليبنتين اكس آر 300 مجم 30 قرص,جراليبنتين 300مجم اقراص,135.0,جراليبنتين 300مجم اقراص
52836,1436,ديباكين 200 مجم / مل شراب 40 مل,ديباكين نقط س ج,34.0,ديباكين نقط س ج
5893,2355,اوفيوسيديك قطرة لزجة للعين 5 مل,اوفيوسيدك قطره 18ج,18.0,اوفيوسيدك قطره 18ج
22777,2570,سيلوكينزوك 200 مجم 28 قرص ممتد المفعول,سيلوكينزوك 200مجم ق,103.0,سيلوكينزوك 200مجم ق
27739,1854,هيموجيت 50 مجم / 5 مل شراب 100 مل,هيموجيت شراب,37.5,هيموجيت شراب
38469,998,ثرومبكس جيل 40 جم,ثرومبكس جيل,52.5,ثرومبكس جيل


In [9]:
# search for @ in the seller_item_name
dataset_df[dataset_df['seller_item_name'].str.contains(".")]

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 cap,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
...,...,...,...,...
83557,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مم ق,39.0
83558,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مل اقراص,39.0
83559,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم مر 30 قرص,39.0
83560,1603,سبانيلا ام آر 35 مجم 30 قرص,سبانيلا 35 مجم ام ار اقراص,39.0


In [10]:
dataset_df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,estohalt 40 mg 14 cap,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [11]:
X = dataset_df['seller_item_name']
y = dataset_df['sku']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
len(X_train), len(X_test)

(66849, 16713)